In [1]:
from pymongo import MongoClient
# from sshtunnel import SSHTunnelForwarder
import pymongo
from tqdm import tqdm
import os
import re

import numpy as np
import pandas as pd

import dateutil.parser

In [2]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
! nvidia-smi

Sun Jun  5 15:54:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 35%   44C    P8    38W / 260W |     22MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 29%   

# MongoDB

In [4]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    # db_de = db.patent_de
    # db_us = db.patent_us
    # db_cn = db.patent_cn
    
    db_rm_us = db.rm_patent_us
    db_rm_cn = db.rm_patent_cn
    db_rm_de = db.rm_patent_de

    db_rm_us_2018 = db.rm_patent_us_2018
    db_rm_cn_2018 = db.rm_patent_cn_2018
    db_rm_de_2018 = db.rm_patent_de_2018
    
except errors.ConnectionFailure as err:
    print(err)

In [5]:
db_rm_us_2018

Collection(Database(MongoClient(host=['140.117.69.70:30241'], document_class=dict, tz_aware=False, connect=True, authmechanism='SCRAM-SHA-1'), 'eva'), 'rm_patent_us_2018')

# US-DE-CN

## 2018

### DE

In [6]:
data_assignee = db_rm_de_2018.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":10}
])

ls_company = []
ls_count = []
for data in data_assignee:
    #print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [7]:
len(ls_company)

113

In [8]:
ls_c = []
ls_family = []
ls_count = []

for c in ls_company:
    data_fam = db_rm_de_2018.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$countrystatus"},
        {"$group" : {"_id" : "$countrystatus", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_fam:
        #print(data)
        ls_c.append(c)
        ls_family.append(data["_id"])
        ls_count.append(data["count"])
        
df_family_de = pd.DataFrame((zip(ls_c, ls_family, ls_count)), columns = ['company', 'country', 'count'])
df_family_de.head()

,company,country,count
0,Intel Corp,DE,39
1,Intel Corp,WO,25
2,Intel Corp,US,24
3,Intel Corp,CN,20
4,Intel Corp,JP,5


In [9]:
df_family_de.shape

(407, 3)

In [10]:
df_family_de.drop(df_family_de[df_family_de["country"]=="WO"].index, inplace=True)
df_family_de.reset_index(drop=True, inplace=True)

In [11]:
# 轉成company-country 的總數表格
table_de = df_family_de.pivot_table(index='company', values='count', columns='country').fillna(0).reset_index()
c_ls_de = [c for c in table_de.columns if re.search('[A-Za-z]', c)]
final_table_de = table_de[c_ls_de]
final_table_de.head()

country,company,AR,AT,AU,BR,CA,CN,CZ,DE,EA,...,GB,IL,JP,KR,MA,RU,SE,SG,TW,US
0,ACO Severin Ahlmann GmbH and Co KG,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Advatec Projects & Co KG GmbH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,Analog Devices Global ULC,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,Apple Inc,0.0,0.0,0.0,0.0,0.0,12.0,0.0,29.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0
4,Atos Information Technology GmbH Austria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
final_table_de.shape

(113, 23)

In [13]:
final_table_de.to_csv('rm_company_country_de_2018.csv',index=0)

final_table存進MongoDB

In [14]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.rm_company_country_de_2018

In [15]:
# title
conn_db_collection.insert_one({"company":final_table_de.columns.tolist()[0],"country":final_table_de.columns.tolist()[1:]})

# 內容
for k in final_table_de.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"country":k[1:]})

### CN

In [16]:
data_assignee = db_rm_cn_2018.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":10}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [17]:
len(ls_company)

2837

In [18]:
ls_c = []
ls_family = []
ls_count = []

for c in ls_company:
    data_fam = db_rm_cn_2018.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$countrystatus"},
        {"$group" : {"_id" : "$countrystatus", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_fam:
        #print(data)
        ls_c.append(c)
        ls_family.append(data["_id"])
        ls_count.append(data["count"])
        
df_family_cn = pd.DataFrame((zip(ls_c, ls_family, ls_count)), columns = ['company', 'country', 'count'])
df_family_cn.head()

,company,country,count
0,Qualcomm Inc,CN,319
1,Qualcomm Inc,WO,314
2,Qualcomm Inc,EP,312
3,Qualcomm Inc,US,307
4,Qualcomm Inc,TW,163


In [19]:
df_family_cn.shape

(4556, 3)

In [20]:
df_family_cn.drop(df_family_cn[df_family_cn["country"]=="WO"].index, inplace=True)
df_family_cn.reset_index(drop=True, inplace=True)

In [21]:
# 轉成company-country 的總數表格
table_cn = df_family_cn.pivot_table(index='company', values='count', columns='country').fillna(0).reset_index()
c_ls_cn = [c for c in table_cn.columns if re.search('[A-Za-z]', c)]
final_table_cn = table_cn[c_ls_cn]
final_table_cn.head()

country,company,AR,AT,AU,BE,BR,CA,CL,CN,CO,...,PH,PL,PT,RO,RU,SE,SG,TW,US,ZA
0,"5G IP HOLDINGS, LLC",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,ABB Schweiz AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
2,ABB Technology AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
3,AI Speech Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ANHUI CHENGFEI FIRE-FIGHT TECHNOLOGY Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
final_table_cn.shape

(2684, 37)

In [23]:
final_table_cn.to_csv('rm_company_country_cn_2018.csv',index=0)

final_table存進MongoDB

In [24]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.rm_company_country_cn_2018

In [25]:
# title
conn_db_collection.insert_one({"company":final_table_cn.columns.tolist()[0],"country":final_table_cn.columns.tolist()[1:]})

# 內容
for k in final_table_cn.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"country":k[1:]})

### US

In [26]:
data_assignee = db_rm_us_2018.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}#,
    #{"$limit":10}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"].replace("))",")"))
    ls_count.append(data["count"])

In [27]:
len(ls_company)

1543

In [28]:
ls_c = []
ls_family = []
ls_count = []

for c in ls_company:
    try:
        data_fam = db_rm_us_2018.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$countrystatus"},
        {"$group" : {"_id" : "$countrystatus", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    except:
        print(c)
    

    for data in data_fam:
        #print(data)
        ls_c.append(c)
        ls_family.append(data["_id"])
        ls_count.append(data["count"])
        
df_family_us = pd.DataFrame((zip(ls_c, ls_family, ls_count)), columns = ['company', 'country', 'count'])
df_family_us.head()

,company,country,count
0,Qualcomm Inc,US,831
1,Qualcomm Inc,WO,731
2,Qualcomm Inc,CN,644
3,Qualcomm Inc,EP,635
4,Qualcomm Inc,TW,301


In [29]:
df_family_us.shape

(4819, 3)

In [30]:
df_family_us.drop(df_family_us[df_family_us["country"]=="WO"].index, inplace=True)
df_family_us.reset_index(drop=True, inplace=True)

In [31]:
# 轉成company-country 的總數表格
table_us = df_family_us.pivot_table(index='company', values='count', columns='country').fillna(0).reset_index()
c_ls_us = [c for c in table_us.columns if re.search('[A-Za-z]', c)]
final_table_us = table_us[c_ls_us]
final_table_us.head()

country,company,AP,AR,AT,AU,BE,BR,CA,CL,CN,...,PT,RO,RU,SE,SG,TN,TR,TW,US,ZA
0,0Chain LLC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2689090 Canada Inc,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2you Io Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,3M Innovative Properties Co,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0
4,3a Logics Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0


In [32]:
final_table_us.shape

(1526, 46)

In [33]:
final_table_us.to_csv('rm_company_country_us_2018.csv',index=0)

final_table存進MongoDB

In [34]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.rm_company_country_us_2018

In [35]:
# title
conn_db_collection.insert_one({"company":final_table_us.columns.tolist()[0],"country":final_table_us.columns.tolist()[1:]})

# 內容
for k in final_table_us.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"country":k[1:]})

### 合併

In [36]:
# concat de cn
df_c = pd.concat([final_table_de,final_table_cn], join='outer')
df_c2 = pd.concat([df_c,final_table_us], join='outer')
df_c2 = df_c2.fillna(0)
#reset index
t = list(df_c2.sort_index(axis=1).columns)
df_c2 = df_c2.reindex(columns=[t[-1]]+t[:-1])
df_c2

country,company,AP,AR,AT,AU,BE,BR,CA,CL,CN,...,PT,RO,RU,SE,SG,TN,TR,TW,US,ZA
0,ACO Severin Ahlmann GmbH and Co KG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,Advatec Projects & Co KG GmbH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Analog Devices Global ULC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
3,Apple Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0
4,Atos Information Technology GmbH Austria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521,Zuna Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1522,Zyter Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
1523,eBay Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1524,eMemory Technology Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [37]:
# 合併重複的公司
df_c2 = df_c2.groupby(df_c2.company, axis=0).sum().reset_index()

In [38]:
df_c2.head()

country,company,AP,AR,AT,AU,BE,BR,CA,CL,CN,...,PT,RO,RU,SE,SG,TN,TR,TW,US,ZA
0,0Chain LLC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2689090 Canada Inc,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2you Io Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,3M Innovative Properties Co,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0
4,3a Logics Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0


In [39]:
df_c2.to_csv('rm_company_country_all_2018.csv',index=0)

In [40]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.rm_company_country_all_2018

In [41]:
# dict_de = {k[0]:k[1:] for k in final_table_de.values.tolist()}

In [42]:
# title
conn_db_collection.insert_one({"company":df_c2.columns.tolist()[0],"country":df_c2.columns.tolist()[1:]})

# 內容
for k in df_c2.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"country":k[1:]})